In [46]:
import os
import sys
import csv
import datetime
import shutil
import random
import glob
import collections
import subprocess
import pandas as pd
import numpy as np

#### Format annotation database: three different types of CG probes.

In [57]:
#
db_dir = "/home/galaxy/project/m6AQTL/data/meqtl/"
os.chdir(db_dir)
file_1, file_2, file_3 = "hm450.hg38.manifest.tsv", "hm27.hg38.manifest.tsv","EPIC.hg38.manifest.gencode.v22.tsv"
df_1 = pd.read_table(file_1, sep="\t")[["probeID", "gene_HGNC", "CpG_chrm", "probeBeg", "probeEnd"]]
df_2 = pd.read_table(file_2, sep="\t")[["probeID", "gene_HGNC", "CpG_chrm", "probeBeg", "probeEnd"]]
# df_3 = pd.read_table(file_3, sep="\t")[["probeID", "geneNames", "CGI"]].dropna(how="any")
# print(df_3.head())
# df_3 = df_3[df_3["CGI"] != "NA"]
# print(df_3.head())
# df_3["CpG_chrm"] = df_3["CGI"].str.split(":").str[1]
# df_3["probeBeg"] = df_3["CGI"].astype(str).str.split(":").str[2].str.split("-").str[0]
# df_3["probeEnd"] = df_3["CGI"].str.split("-").str[1]
# df_3["gene_HGNC"] = df_3["geneNames"]
# df_3 = df_3[["probeID", "gene_HGNC", "CpG_chrm", "probeBeg", "probeEnd"]]
df = pd.concat([df_1, df_2]).drop_duplicates().sort_values(["probeID"]) # df_3
df["probeEnd"] = df["probeEnd"].astype(int)
# df.to_csv("hg450-hg27.hg38.mainfest.tsv", sep="\t", index=False)
anno_dict = {}
df_db = df
for i, values in df_db.iterrows():
    feature_id = "%s:%s-%s" % (values["CpG_chrm"], str(values["probeBeg"]), str(int(values["probeEnd"])))
    anno_dict[values["probeID"]] = {"Gene": values["gene_HGNC"], "ID": feature_id}

#### annotate meQTL based on db;

In [67]:
meqtl = "/home/lzg/1906/QTL_data/unfiltered/get_meQTL/meQTL_db_standard.txt"
#####################################################################################
df_meqtl = pd.read_table(meqtl, sep="\t", skiprows=1)
gene_list, id_list = [], []
probe_list = [str(x).strip() for x in df_meqtl["Probe_Name"].tolist()]
for i in range(len(probe_list)):
    probe = probe_list[i]
    if probe in anno_dict:
        gene_list.append(anno_dict[probe]["Gene"])
        id_list.append(anno_dict[probe]["ID"])
    else:
#         gene_list.append(df_meqtl.iloc[i, 1]) # keep the old records.
#         id_list.append(df_meqtl.iloc[i, 2])
        gene_list.append(np.nan) # replace the old records with np.nan.
        id_list.append(np.nan)
df_meqtl["Ensembl_ID"],df_meqtl["Gene_Name"],df_meqtl["ID"]  = np.nan, gene_list, id_list
print(df_meqtl.head())

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   Ensembl_ID      Gene_Name                        ID  Probe_Name  \
0         NaN           HOPX    chr4:56657611-56657660  cg19978674   
1         NaN  CYREN;TMEM140  chr7:135164832-135164881  cg07876788   
2         NaN         UBAP2L  chr1:154265790-154265839  cg14859874   
3         NaN          BTNL9  chr5:181051136-181051185  cg00299286   
4         NaN       ARHGAP25    chr2:68816329-68816378  cg23817627   

        RS_ID REF ALT meQTL_type Cutoff         Pvalue Corrected_pvalue  \
0   rs7664589   C   T        cis  250kb  3.270000e-310                0   
1  rs11970825   C   T        cis  250kb  3.270000e-310                0   
2   rs1194596   G   A        cis  250kb  3.270000e-310                0   
3   rs9686314   A   G        cis  250kb  3.270000e-310                0   
4   rs6546439   G   A        cis  250kb  3.270000e-310                0   

  Measure     Value    Cell_or_tissue  Condition  Experimental_tag  PubMed_ID  
0    Beta  0.917694  peripheral blood        NaN

In [68]:
# filter out ID(null) rows.
print("\t%d" % len(df_meqtl))
df_meqtl = df_meqtl[~df_meqtl["ID"].isnull()]
print("\t%d" % len(df_meqtl))

	95814
	95814


In [69]:
result_list = []
for i, values in df_meqtl.iterrows():
    gene, i_list = str(values["Gene_Name"]), values.tolist()
    for x in list(set(gene.split(";"))):
        i_list[1] = x
        result_list.append(i_list)
df_result = pd.DataFrame(result_list, columns=df_meqtl.columns).drop_duplicates()
df_result.to_csv("/home/galaxy/project/m6AQTL/data/meqtl/total_data/meQTL_db.txt", sep="\t", index=False)

In [63]:
# can't find "cg080995532" which position endswith NCBI build 36 in hm27.hg38.manifest.tsv;
print("\t%d" % len(df_result))
df_result = df_result[~df_result["ID"].str.contains("NCBI build 36", na=False)]
print("\t%d" % len(df_result))
# can't find "ch_11_110329410F" which position endswith hg19 in hm450.hg38.manifest.tsv;
print("\t%d" % len(df_result))
df_result = df_result[~df_result["Probe_Name"].str.startswith("ch_", na=False)]
print("\t%d" % len(df_result))

95814
95814


#### liftover

In [65]:
print(list(set([str(x).split(":")[-1] for x in df_result["ID"].tolist() if len(str(x).split(":"))>2])))

[]


In [54]:
# ### output hg19
df_hg19 = df_result[df_result["ID"].str.contains("hg19", na=False)]
print("\t%d" % len(df_hg19))
id_list = df_hg19["ID"].tolist()
bed_list = []
for x in id_list:
    if len(x.split(":")[1].split("-")) < 2:
        print(x)
    else:
        bed_list.append("\t".join([x.split(":")[0], x.split(":")[1].split("-")[0], x.split(":")[1].split("-")[1], x]))
content = "\n".join(bed_list)
hg19_bed = "/home/galaxy/project/m6AQTL/data/meqtl/total_data/liftover/hg19.bed"
with open(hg19_bed, 'w') as fw:
    fw.write(content)
# ### liftover
chain_file, failure_file = "/data/database/chain_file/hg19ToHg38.over.chain", "/home/galaxy/failure.txt"
hg38_bed = "/home/galaxy/project/m6AQTL/data/meqtl/total_data/liftover/hg38.bed"
command = "liftOver %s %s %s %s" % (hg19_bed, chain_file, hg38_bed, failure_file)
os.system(command)

In [55]:
#### Replace hg19 ID with hg38 ID;
df_hg38 = df_result[~df_result["ID"].str.contains("hg19", na=False)]
print("\t%d" % len(df_hg38))
######
hg38_bed = "/home/galaxy/project/m6AQTL/data/meqtl/total_data/liftover/hg38.bed"
df_bed = pd.read_table(hg38_bed, sep="\t", header=None)
df_bed["hg38_ID"] = df_bed.iloc[:, 0] + ":" + df_bed.iloc[:, 1].astype(str) + "-" + df_bed.iloc[:, 2].astype(str)
# print(df_bed.head())
hg_dict = dict(zip(df_bed.iloc[:, 3], df_bed["hg38_ID"]))
#
hg38_list = [hg_dict[x] if x in hg_dict else np.nan for x in df_hg19["ID"].tolist()]
df_hg19["ID"] = hg38_list
df_hg19 = df_hg19[~df_hg19["ID"].str.contains("hg19", na=False)]
#
df_final = pd.concat([df_hg38, df_hg19]).sort_values(["PubMed_ID"])
df_final.to_csv("/home/galaxy/project/m6AQTL/data/meqtl/total_data/meQTL_paper.txt", sep="\t", index=False)

	284475


/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


#### add Ensembl ID 

In [73]:
file_1 = "/home/galaxy/project/m6AQTL/data/meqtl/total_data/meQTL_db.txt"
file_2 = "/home/galaxy/project/m6AQTL/data/meqtl/total_data/meQTL_paper.txt"
gene_type = "/home/lzg/1906/QTL_data/unused_qtl_database/eqtl/no_genename_or_ensgid/gene_type.txt"
df = pd.read_table(gene_type, sep="\t", header=None)
df.columns = ["Ensembl_ID", "Gene_Name", "Gene_Type"]
df["New_Ensembl"] = [x.split(".")[0] for x in df["Ensembl_ID"].tolist()]
type_dict = dict(zip(df["Gene_Name"], df["New_Ensembl"]))
######
for x in [file_1, file_2]:
    df_i = pd.read_table(x, sep="\t")
    name_list = df_i["Gene_Name"].tolist()
    ensembl_list = [type_dict[name] if name in type_dict else np.nan for name in name_list]
    df_i["Ensembl_ID"] = ensembl_list
    df_i.to_csv(x.replace(".txt", "_with_Ensembl.txt"), sep="\t", index=False)

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,8,10,11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
